## Naive bayes classifier
### Imports

In [1]:
import pandas as pd 
import numpy as np

In [2]:
train_features_df = pd.read_csv("Data/train-features.txt", sep=" ", header=None, names=["d", "s", "n_s_in_d"])
train_labels_df = pd.read_csv("Data/train-labels.txt", sep=" ", header=None, names=["label"])

train_features_50_df = pd.read_csv("Data/train-features-50.txt", sep=" ", header=None, names=["d", "s", "n_s_in_d"])
train_labels_50_df = pd.read_csv("Data/train-labels-50.txt", sep=" ", header=None, names=["label"])

train_features_100_df = pd.read_csv("Data/train-features-100.txt", sep=" ", header=None, names=["d", "s", "n_s_in_d"])
train_labels_100_df = pd.read_csv("Data/train-labels-100.txt", sep=" ", header=None, names=["label"])

train_features_400_df = pd.read_csv("Data/train-features-400.txt", sep=" ", header=None, names=["d", "s", "n_s_in_d"])
train_labels_400_df = pd.read_csv("Data/train-labels-400.txt", sep=" ", header=None, names=["label"])


test_features_df = pd.read_csv("Data/test-features.txt", sep=" ", header=None, names=["d", "s", "n_s_in_d"])
test_labels_df = pd.read_csv("Data/test-labels.txt", sep=" ", header=None, names=["label"])

In [3]:
train_features_df = train_features_df.pivot(index='d', columns='s', values='n_s_in_d')
train_features_df = train_features_df.fillna(0)

train_features_50_df = train_features_50_df.pivot(index='d', columns='s', values='n_s_in_d')
train_features_50_df = train_features_50_df.fillna(0)

train_features_100_df = train_features_100_df.pivot(index='d', columns='s', values='n_s_in_d')
train_features_100_df = train_features_100_df.fillna(0)

train_features_400_df = train_features_400_df.pivot(index='d', columns='s', values='n_s_in_d')
train_features_400_df = train_features_400_df.fillna(0)

test_features_df = test_features_df.pivot(index='d', columns='s', values='n_s_in_d')
test_features_df = test_features_df.fillna(0)

In [4]:
train_features_df

s,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
d,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
698,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Classifier implementation

In [5]:
class NBC:
    def __init__(self):
        self.p_y_1 = None
        self.p_y_0 = None
        self.p_X = None
        self.y_pred = None
    
    def fit(self, X_, y_):
        X = np.array(X_)
        [m, n] = X.shape
        y = y_['label']
        y1_idx = np.where(y == 1)
        y0_idx = np.where(y == 0)
        self.p_y_1 = len(y1_idx[0]) / (len(y0_idx[0]) + len(y1_idx[0]))
        self.p_y_0 = len(y0_idx[0]) / (len(y0_idx[0]) + len(y1_idx[0]))
        p_X_1 = np.zeros(n)
        p_X_0 = np.zeros(n)
        for i in range(n):
            p_X_1[i] = (X[y1_idx, i].sum() + 1) / (X.sum(axis=1)[y1_idx].sum() + X.sum())
            p_X_0[i] = (X[y0_idx, i].sum() + 1) / (X.sum(axis=1)[y0_idx].sum() + X.sum())
        self.p_X = pd.DataFrame(data=[p_X_1, p_X_0], columns = X_.columns, index = ['1', '0'])
        
    def predict(self, X_):
        common_cols = self.p_X.columns.intersection(X_.columns)
        log_p_y_1 = np.log(self.p_y_1)
        log_p_y_0 = np.log(self.p_y_0)

        log_p_X = np.log(self.p_X)
        X = np.array(X_)
        [m, n] = X.shape
        self.y_pred = np.zeros(m)
        for i in range(m):
            p_X_1_pred = log_p_y_1
            p_X_0_pred = log_p_y_0
            for ifeature, feature in enumerate(X_.columns[np.where(X[i] != 0)]):
                if feature in common_cols:
                    ifeature_train = np.where(self.p_X.columns == feature)[0][0]
                    p_X_1_pred += log_p_X.iat[0, ifeature_train]
                    p_X_0_pred += log_p_X.iat[1, ifeature_train]
            self.y_pred[i] = (p_X_1_pred > p_X_0_pred) * 1 
            
    def score(self, y_test):
        unique, counts = np.unique(np.array(y_test['label']) == self.y_pred, return_counts=True)
        return counts[1] / (counts.sum())

### Learning and testing a classifier on a base set
Prediction for learning set

In [6]:
clf = NBC()
clf.fit(train_features_df, train_labels_df)
clf.predict(train_features_df)
print(f"Accuracy: {np.round(clf.score(train_labels_df)*100, 3)}%")

Accuracy: 99.286%


Prediction for the test set

In [7]:
clf.predict(test_features_df)
print(f"Accuracy: {np.round(clf.score(test_labels_df)*100, 3)}%")

Accuracy: 97.692%


### Learning and testing the classifier on smaller sets
A set with count 50

In [8]:
clf = NBC()
clf.fit(train_features_50_df, train_labels_50_df)
clf.predict(test_features_df)
print(f"Accuracy: {np.round(clf.score(test_labels_df)*100, 5)}%")

Accuracy: 67.30769%


A set with count 100

In [9]:
clf = NBC()
clf.fit(train_features_100_df, train_labels_100_df)
clf.predict(test_features_df)
print(f"Accuracy: {np.round(clf.score(test_labels_df)*100, 5)}%")

Accuracy: 94.23077%


A set with count 400

In [10]:
clf = NBC()
clf.fit(train_features_400_df, train_labels_400_df)
clf.predict(test_features_df)
print(f"Accuracy: {np.round(clf.score(test_labels_df)*100, 5)}%")

Accuracy: 96.92308%


### Learning and testing a classifier on sets with disproportionate distribution

In [11]:
for n in range(1, 4):
    idxs = [np.random.randint(1, train_features_df.shape[0]/2) for i in range(n*100)]
    train_features_df_reduced = train_features_df.drop(idxs)
    train_labels_df_reduced = train_labels_df.drop(idxs)
    unique, counts = np.unique(train_labels_df_reduced, return_counts=True)
    part = int(counts[0]*100/(counts.sum()))
    print(f"Proportion: {part}% to {100-part}%")
    print(f"Number of samples: {train_labels_df_reduced.shape[0]}")
    clf = NBC()
    clf.fit(train_features_df_reduced, train_labels_df_reduced)
    clf.predict(test_features_df)
    print(f"Accuracy: {np.round(clf.score(test_labels_df)*100, 5)}% \n")

Proportion: 43% to 57%
Number of samples: 615
Accuracy: 95.76923% 

Proportion: 36% to 64%
Number of samples: 549
Accuracy: 91.53846% 

Proportion: 31% to 69%
Number of samples: 509
Accuracy: 89.23077% 

